# Лабораторная работа 1. Перцептрон

**Задание**: реализовать граничное условие для окончания обучения перцептрона с использованием правила сходимости и зацикливания

## Загрузка модулей

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Загрузка и подготовка данных

In [48]:
df = pd.read_csv('data.csv')

df = df.iloc[np.random.permutation(len(df))]
y = df.iloc[0:100, 4].values
y = np.where(y == "Iris-setosa", 1, -1)
X = df.iloc[0:100, [0, 2]].values
df.head(10)

,0,1,2,3,4
92,5.8,2.6,4.0,1.2,Iris-versicolor
44,5.1,3.8,1.9,0.4,Iris-setosa
136,6.3,3.4,5.6,2.4,Iris-virginica
29,4.7,3.2,1.6,0.2,Iris-setosa
64,5.6,2.9,3.6,1.3,Iris-versicolor
74,6.4,2.9,4.3,1.3,Iris-versicolor
84,5.4,3.0,4.5,1.5,Iris-versicolor
134,6.1,2.6,5.6,1.4,Iris-virginica
94,5.6,2.7,4.2,1.3,Iris-versicolor
83,6.0,2.7,5.1,1.6,Iris-versicolor


## Параметры модели

In [49]:
inputSize = X.shape[1] # количество входных сигналов равно количеству признаков задачи 
hiddenSizes = 10 # число нейронов скрытого (А) слоя 
outputSize = 1 if len(y.shape) else y.shape[1] # количество выходных сигналов равно количеству классов задачи


# матрица весов скрытого слоя
Win = np.zeros((1+inputSize,hiddenSizes)) 
# пороги w0 задаем случайными числами
Win[0,:] = (np.random.randint(0, 3, size = (hiddenSizes))) 
# остальные веса задаем случайно -1, 0 или 1 
Win[1:,:] = (np.random.randint(-1, 2, size = (inputSize,hiddenSizes))) 

# случайно инициализируем веса выходного слоя
Wout = np.random.randint(0, 2, size = (1+hiddenSizes,outputSize)).astype(np.float64)

## Функция предсказания

In [50]:
def predict(Xp):
    # выходы первого слоя = входные сигналы * веса первого слоя
    hidden_predict = np.where((np.dot(Xp, Win[1:,:]) + Win[0,:]) >= 0.0, 1, -1).astype(np.float64)
    # выходы второго слоя = выходы первого слоя * веса второго слоя
    out = np.where((np.dot(hidden_predict, Wout[1:,:]) + Wout[0,:]) >= 0.0, 1, -1).astype(np.float64)
    return out, hidden_predict

## Обучение с фиксированным количеством итераций

In [51]:
n_iter=5
eta = 0.01
for i in range(n_iter):
    for xi, target, j in zip(X, y, range(X.shape[0])):
        pr, hidden = predict(xi) 
        Wout[1:] += ((eta * (target - pr)) * hidden).reshape(-1, 1)
        Wout[0] += eta * (target - pr)

## Обучение с граничным условием о сходимости и зацикливании

In [52]:
import numpy as np
# Задаем скорость обучения (learning rate)
eta = 0.01
# Переменные для отслеживания сходимости
converged = False
no_change_count = 0

while not converged:
     # Список для хранения весов на каждой эпохе
    epoch_weights = []

    for xi, target in zip(X, y):
        pr, hidden = predict(xi)
        # Вычисляем ошибку
        error = target - pr
         # Обновляем веса для выходного слоя
        Wout[1:] += ((eta * (target - pr)) * hidden).reshape(-1, 1)
        Wout[0] += eta * (target - pr)
        
    # Проверяем, все ли примеры правильно классифицированы
    all_correct = np.all([predict(xi)[0] == target for xi, target in zip(X, y)])


    if all_correct:
        converged = True
        break
# Если веса не изменились, это может быть признаком зацикливания
    if no_change_count > 0:
        print('Повтор весов')
        break

    weight = np.sort(Wout.copy().flatten())
    epoch_weights.append(weight)
 # Проверяем, были ли такие же веса ранее
    if any(np.array_equal(w, weight) for w in epoch_weights):
        print(epoch_weights)
        no_change_count += 1
        break

if converged:
    print('Отсутствуют ошибки')

[array([-0.62, -0.38, -0.34,  0.38,  0.38,  0.38,  0.38,  0.38,  0.62,
        0.62,  0.62])]


## Проверка модели на работу

In [53]:
y = df.iloc[:, 4].values
y = np.where(y == "Iris-setosa", 1, -1)
X = df.iloc[:, [0, 2]].values
pr, hidden = predict(X)
sum(pr-y.reshape(-1, 1))

array([14.])

In [54]:
Wout.reshape(1, -1)[0].tolist()

[0.37999999999999945,
 0.6200000000000002,
 -0.6200000000000002,
 0.6200000000000002,
 -0.34000000000000064,
 0.37999999999999945,
 -0.38000000000000006,
 0.37999999999999945,
 0.37999999999999945,
 0.37999999999999945,
 0.6199999999999997]

In [55]:
epoch_weights

[array([-0.62, -0.38, -0.34,  0.38,  0.38,  0.38,  0.38,  0.38,  0.62,
         0.62,  0.62])]

In [56]:
epoch_weights

[array([-0.62, -0.38, -0.34,  0.38,  0.38,  0.38,  0.38,  0.38,  0.62,
         0.62,  0.62])]

In [57]:
t = [[-0.34, -0.34, -0.22,  0.66,  0.34, -0.34,  0.66,  1.34,  0.66,
        0.66, -0.34]]

[-0.34, -0.34, -0.22,  0.66,  0.34, -0.34,  0.66,  1.34,  0.66, 0.66, -0.34] in t

True